In [1]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
!pip install tqdm
!pip install -U -q PyDrive

Selecting previously unselected package libxext6:amd64.
(Reading database ... 16669 files and directories currently installed.)
Preparing to unpack .../libxext6_2%3a1.3.3-1_amd64.deb ...
Unpacking libxext6:amd64 (2:1.3.3-1) ...
Selecting previously unselected package x11-common.
Preparing to unpack .../x11-common_1%3a7.7+19ubuntu3_all.deb ...
Unpacking x11-common (1:7.7+19ubuntu3) ...
Selecting previously unselected package libice6:amd64.
Preparing to unpack .../libice6_2%3a1.0.9-2_amd64.deb ...
Unpacking libice6:amd64 (2:1.0.9-2) ...
Selecting previously unselected package libsm6:amd64.
Preparing to unpack .../libsm6_2%3a1.2.2-1_amd64.deb ...
Unpacking libsm6:amd64 (2:1.2.2-1) ...
Setting up libxext6:amd64 (2:1.3.3-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up x11-common (1:7.7+19ubuntu3) ...
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#   print('title: %s, id: %s' % (file1['title'], file1['id']))

#2. Get the file
downloaded = drive.CreateFile({'id':'1GRWv50mpTON7ZoybA7-RoxBQ4AgrjnOB'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.zip') 

#2. Get the file
downloaded = drive.CreateFile({'id':'1J_bWRKyUYmBmJAhxaV1xjDn7VTsinvLO'}) # replace the id with id of file you want to access
downloaded.GetContentFile('test.zip')  


In [1]:
!ls

datalab  res.csv  test	test.zip  train  train.zip


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('./train.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()
zip_ref = zipfile.ZipFile('./test.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [6]:
import re
import os
TRAIN_DIR = './train/'
TEST_DIR = './test/'
ordered_files = sorted(os.listdir(TEST_DIR), key=lambda x: (int(re.sub('\D','',x)),x))
test_images =  [TEST_DIR+i for i in ordered_files]
print(len(test_images))

12500


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.contrib.layers import dropout
import csv
import time
import re

TRAIN_DIR = './train/'
TEST_DIR = './test/'

IMAGE_SIZE = 200;
CHANNELS = 3

valid_size = 50
pixel_depth = 255.0

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

def read_image(file_path):
  img = cv2.imread(file_path, cv2.IMREAD_COLOR)
  if (img.shape[0] >= img.shape[1]):
    resizeto = (IMAGE_SIZE, int(round(IMAGE_SIZE * (float(img.shape[1]) / img.shape[0]))))
  else:
    resizeto = (int(round(IMAGE_SIZE * (float(img.shape[0])  / img.shape[1]))), IMAGE_SIZE)
  img = cv2.resize(img, (resizeto[1], resizeto[0]), interpolation=cv2.INTER_CUBIC)
#   img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_CUBIC)
  img = cv2.copyMakeBorder(img, 0, IMAGE_SIZE - img.shape[0], 0,
    IMAGE_SIZE - img.shape[1], cv2.BORDER_CONSTANT, 0)
  return img[:,:,::-1]

def prep_images(images, normalize=True):
  count = len(images)
  data = np.ndarray((count, IMAGE_SIZE, IMAGE_SIZE, CHANNELS), dtype=np.float32)
#   for i in tqdm(range(count)):
  for i in range(count):
    image = read_image(images[i])
    image_data = np.array(image, dtype=np.float32)
    if normalize:
      image_data[:,:,0] = image_data[:,:,0].astype(float) / pixel_depth
      image_data[:,:,1] = image_data[:,:,1].astype(float) / pixel_depth
      image_data[:,:,2] = image_data[:,:,2].astype(float) / pixel_depth
    data[i] = image_data
  return data

def randomize_data(data):
  data = np.array(data)
  permutation = np.random.permutation(data.shape[0])
  shuffled_data = data[permutation]
  return shuffled_data

train_images = randomize_data(train_images)

# train_labels = np.array([1 if 'dog' in i else 0 for i in train_images[:3000]])
# train_normalized = prep_images(train_images[:3000])

print("Train dataset shape: {}".format(train_images.shape))

# plt.imshow(train_normalized[0,:,:,:])
# plt.show()

# train_dataset, train_labels = train_normalized, train_labels

def create_val_set(data, labels, valid_size):
  data_size = data.shape[0]
  all_indices = np.array((range(data_size)))
  val_indices = np.random.choice(range(data_size), valid_size, replace=False)
  train_indices = np.delete(all_indices, val_indices)
  data_train = np.delete(data, val_indices, axis=0)
  label_train = np.delete(labels, val_indices, axis=0)
  data_val = np.delete(data, train_indices, axis=0)
  label_val = np.delete(labels, train_indices, axis=0)
  return data_train, label_train, data_val, label_val

train_dataset, train_labels,\
valid_dataset, valid_labels = create_val_set(train_images, train_images, valid_size)

print('Train dataset shape: {}'.format(train_dataset.shape))
print('Train labels shape: {}'.format(train_labels.shape))
print('Validation dataset shape: {}'.format(valid_dataset.shape))
print('Validation labels shape: {}'.format(valid_labels.shape))

learning_rate = 0.001
conv_keep_prob = 0.9
fc_keep_prob = 0.5

num_classes= 2

X = tf.placeholder(tf.float32,
  shape=(None, IMAGE_SIZE, IMAGE_SIZE, CHANNELS))
y = tf.placeholder(tf.int32, shape=(None)) # 1 - dog, 0 - cat
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')

def create_conv(x, kernel_shape, stride, name):
  with tf.name_scope(name):
    conv_kernel_init = tf.truncated_normal(kernel_shape, dtype=tf.float32, stddev=0.1)
    conv_kernel = tf.Variable(conv_kernel_init, name='weights_'+name)
    conv_bias = tf.Variable(tf.constant(0.0, shape=[kernel_shape[-1]], dtype=tf.float32),
      trainable=True, name='biases_'+name)
    conv = tf.nn.conv2d(x, conv_kernel, stride, padding='SAME')
    out = tf.nn.relu(tf.nn.bias_add(conv, conv_bias))
    return out

conv1 = create_conv(X, [3, 3, 3, 64], [1, 1, 1, 1], 'conv1')
conv1_1 = create_conv(conv1, [3, 3, 64, 64], [1, 1, 1, 1], 'conv1_1')
pool1 = tf.nn.max_pool(conv1_1,
  ksize=[1, 2, 2, 1],
  strides=[1, 2, 2, 1],
  padding='SAME',
  name='pool1')
# drop1 = dropout(pool1, conv_keep_prob, is_training=is_training)
conv2 = create_conv(pool1, [3, 3, 64, 128], [1, 1, 1, 1], 'conv2')
conv2_2 = create_conv(conv2, [3, 3, 128, 128], [1, 1, 1, 1], 'conv2_2')
pool2 = tf.nn.max_pool(conv2_2,
  ksize=[1, 2, 2, 1],
  strides=[1, 2, 2, 1],
  padding='SAME',
  name='pool2')
# drop2 = dropout(pool2, conv_keep_prob, is_training=is_training)
conv3 = create_conv(pool2, [3, 3, 128, 256], [1, 1, 1, 1], 'conv3')
conv3_1 = create_conv(conv3, [3, 3, 256, 256], [1, 1, 1, 1], 'conv3_1')
conv3_2 = create_conv(conv3_1, [3, 3, 256, 256], [1, 1, 1, 1], 'conv3_2')
pool3 = tf.nn.max_pool(conv3_2,
  ksize=[1, 2, 2, 1],
  strides=[1, 2, 2, 1],
  padding='SAME',
  name='pool3')
# drop3 = dropout(pool3, conv_keep_prob, is_training=is_training)
conv4 = create_conv(pool3, [3, 3, 256, 512], [1, 1, 1, 1], 'conv4')
conv4_1 = create_conv(conv4, [3, 3, 512, 512], [1, 1, 1, 1], 'conv4_1')
conv4_2 = create_conv(conv4_1, [3, 3, 512, 512], [1, 1, 1, 1], 'conv4_2')
pool4 = tf.nn.max_pool(conv4_2,
  ksize=[1, 2, 2, 1],
  strides=[1, 2, 2, 1],
  padding='SAME',
  name='pool4')
# drop4 = dropout(pool4, conv_keep_prob, is_training=is_training)
conv5 = create_conv(pool4, [3, 3, 512, 512], [1, 1, 1, 1], 'conv5')
conv5_1 = create_conv(conv5, [3, 3, 512, 512], [1, 1, 1, 1], 'conv5_1')
conv5_2 = create_conv(conv5_1, [3, 3, 512, 512], [1, 1, 1, 1], 'conv5_2')
pool5 = tf.nn.max_pool(conv5_2,
  ksize=[1, 2, 2, 1],
  strides=[1, 2, 2, 1],
  padding='SAME',
  name='pool5')
# drop5 = dropout(pool5, conv_keep_prob, is_training=is_training)
# conv6 = create_conv(drop5, [3, 3, 128, 128], [1, 1, 1, 1], 'conv6')
# pool6 = tf.nn.max_pool(conv6,
#   ksize=[1, 2, 2, 1],
#   strides=[1, 2, 2, 1],
#   padding='SAME',
#   name='pool6')
# drop6 = dropout(pool6, conv_keep_prob, is_training=is_training)

def create_fully_connected(x, neurons, name, activation=None):
  with tf.name_scope(name):
    n_inputs = int(x.get_shape()[1])
    stddev = np.sqrt(2)*np.sqrt(2.0/(n_inputs+neurons))
    fc_init = tf.truncated_normal((n_inputs, neurons), dtype=tf.float32, stddev=stddev)
    fc_W =  tf.Variable(fc_init, name='fc_weights_'+name)
    fc_b = tf.Variable(tf.constant(0.0, shape=[neurons], dtype=tf.float32),
      trainable=True, name='fc_biases_'+name)
    z = tf.nn.bias_add(tf.matmul(x, fc_W), fc_b)
    if activation == 'relu':
      return tf.nn.relu(z)
    else:
      return z

shape = int(np.prod(pool5.get_shape()[1:]))
x_flat = tf.reshape(pool5, [-1, shape])

fc1 = create_fully_connected(x_flat, 4096, 'fc1', activation='relu')
fcdrop1 = dropout(fc1, fc_keep_prob, is_training=is_training)
fc2 = create_fully_connected(fcdrop1, 4096, 'fc2', activation='relu')
fcdrop2 = dropout(fc2, fc_keep_prob, is_training=is_training)
# fc3 = create_fully_connected(fcdrop2, 128, 'fc3', activation='relu')
# fcdrop3 = dropout(fc3, fc_keep_prob, is_training=is_training)
y_pred = create_fully_connected(fcdrop2, num_classes, 'fc_out')
prob_pred = tf.nn.softmax(y_pred)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(y_pred, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

def fetch_batch(x, y, batch_index, batch_size):
  start = batch_index*batch_size
  end = batch_index*batch_size+batch_size
  x_batch = x[start:end]
  if y is not None:
    y_batch = y[start:end]
    return x_batch, y_batch
  else:
    return x_batch

n_epoches = 20
batch_size = 50
n_batches = int(np.ceil(train_dataset.shape[0] / batch_size))
print('Training settings:\n\tepoches: {}\n\tbatch size:{}\n\tbathes: {}'.format(n_epoches,
  batch_size, n_batches))

valid_dataset = prep_images(valid_dataset)
valid_labels = np.array([1 if 'dog' in i else 0 for i in valid_labels])
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epoches):
    for batch_index in range(n_batches):
      if batch_index%100==0:
        print(batch_index, '/', n_batches)
      x_batch_imgs, label_batch_imgs = fetch_batch(train_dataset, train_labels,
        batch_index, batch_size)
      x_batch = prep_images(x_batch_imgs)
      label_batch = np.array([1 if 'dog' in i else 0 for i in label_batch_imgs])
      sess.run(training_op, feed_dict={X: x_batch, y: label_batch, is_training:True})
    if epoch%1 == 0:
      loss_value = sess.run(loss, feed_dict={X: x_batch, y: label_batch, is_training:False})
      accuracy_value = sess.run(accuracy, feed_dict={X: x_batch, y: label_batch, is_training:False})
      print('Epoch: {}, batch accuracy: {:.1f}%, batch loss: {:.5f}'\
        .format(epoch, (accuracy_value*100), loss_value))
      accuracy_valid = sess.run(accuracy, feed_dict={X: valid_dataset, y: valid_labels, is_training:False})
      print('Epoch: {}, validation accuracy: {:.1f}%'\
        .format(epoch, (accuracy_valid*100)))
    
  print('Training has finished')

  time.sleep(2)
  ordered_files = sorted(os.listdir(TEST_DIR), key=lambda x: (int(re.sub('\D','',x)),x))
  test_images =  [TEST_DIR+i for i in ordered_files]
  test_normalized = prep_images(test_images)
  print("Test shape: {}".format(test_normalized.shape))
  test_batches = int(np. ceil(test_normalized.shape[0] / batch_size))
  label_pred = np.empty((0,2), np.float32)
  for batch_index in range(test_batches):
    x_batch = fetch_batch(test_normalized, None,
      batch_index, batch_size)
    label_pred = np.append(label_pred, sess.run(prob_pred, feed_dict={X: x_batch, is_training:False}), axis=0)
  print(label_pred.shape)
  print(label_pred[:20])
  with open('./res.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(['id', 'label'])
    for i, v in enumerate(label_pred):
      csvwriter.writerow([i+1, v[1]])

Train dataset shape: (25000,)
Train dataset shape: (24950,)
Train labels shape: (24950,)
Validation dataset shape: (50,)
Validation labels shape: (50,)
Training settings:
	epoches: 20
	batch size:50
	bathes: 499
0 / 499
100 / 499
200 / 499
300 / 499
400 / 499
Epoch: 0, batch accuracy: 56.0%, batch loss: 0.66162
Epoch: 0, validation accuracy: 42.0%
0 / 499
100 / 499
200 / 499
300 / 499
400 / 499
Epoch: 1, batch accuracy: 62.0%, batch loss: 0.65659
Epoch: 1, validation accuracy: 34.0%
0 / 499
100 / 499
200 / 499
300 / 499


KeyboardInterrupt: ignored

In [0]:
from google.colab import files

files.download('./res.csv')